In [2]:
# ================================
# HyRA-CXR Ablation (No-Attention / No-Residual)
# Reads best HPs from outputs/kt_holdout_trials.csv
# Saves all results under outputs2/
# ================================

import os,cv2, json, gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from keras_tuner import HyperParameters

# ---------- 0) إعدادات عامة (نقرأ من بيئتك؛ مع قيم افتراضية لو غير معرفة) ----------
SEED       = globals().get("SEED", 42)
BATCH_SIZE = globals().get("BATCH_SIZE", 16)
FOLDS      = globals().get("FOLDS", 5)
img_size   = globals().get("img_size", 224)
categories = globals().get("categories", ["Normal", "Lung_Opacity", "Viral_Pneumonia"])

np.random.seed(SEED)
tf.random.set_seed(SEED)

# ======================
# تحميل الصور (كما في كودك الأصلي)
# ======================
base_dir   = "D://Lung X-Ray Image"
categories = ["Normal", "Lung_Opacity", "Viral_Pneumonia"]
img_size   = 224

data, labels = [], []
for idx, category in enumerate(categories):
    cat_dir = os.path.join(base_dir, category)
    for fname in os.listdir(cat_dir):
        fpath = os.path.join(cat_dir, fname)
        img = cv2.imread(fpath)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size, img_size))
        data.append(img)
        labels.append(idx)

data   = np.array(data, dtype="float32") / 255.0
labels = np.array(labels, dtype="int32")

print(f"Loaded: {data.shape}, Labels: {labels.shape}")


# تحقق من وجود البيانات
assert "data" in globals() and "labels" in globals(), " يرجى التأكد من تعريف data و labels قبل تشغيل هذا الكود."

# ---------- 1) دالة تحميل أفضل HPs من ملف KerasTuner الأصلي ----------
def load_best_hp_from_csv(csv_path="outputs/kt_holdout_trials.csv"):
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"Could not find: {csv_path}")
    df = pd.read_csv(csv_path)
    # تأكد من وجود عمود الدقة
    if "val_accuracy_best" not in df.columns:
        raise ValueError("CSV must contain 'val_accuracy_best' column.")
    # اختَر الصف الأفضل
    df = df.sort_values("val_accuracy_best", ascending=False).reset_index(drop=True)
    row = df.iloc[0]
    hp = HyperParameters()
    hp.values = {
        "l2_weight": float(row["hp_l2_weight"]),
        "dropout": float(row["hp_dropout"]),
        "filters_s1": int(row["hp_filters_s1"]),
        "filters_s2": int(row["hp_filters_s2"]),
        "filters_s3": int(row["hp_filters_s3"]),
        "head_filters": int(row["hp_head_filters"]),
        "optimizer": str(row["hp_optimizer"]),
        "lr": float(row["hp_lr"]),
    }
    return hp, float(row["val_accuracy_best"])

best_hp, holdout_valacc = load_best_hp_from_csv("outputs/kt_holdout_trials.csv")
print("Loaded best HPs from CSV:", best_hp.values)
print(f"Best holdout val_accuracy (from CSV): {holdout_valacc:.4f}")

# ---------- 2) Augmentation + Dataset pipeline (نستخدم نفس make_ds إن كانت معرّفة) ----------
if "make_ds" not in globals():
    # نفس سلوك كودك الأصلي اختصارًا
    augment = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.08),
        layers.RandomZoom(0.10),
        layers.RandomTranslation(0.05, 0.05),
        layers.RandomContrast(0.10),
    ], name="augment_pipeline")

    def make_ds(x, y, batch, training=False, seed=SEED):
        x = np.ascontiguousarray(x, dtype=np.float32)
        y = np.ascontiguousarray(y, dtype=np.float32)
        with tf.device('/CPU:0'):
            ds = tf.data.Dataset.from_tensor_slices((x, y))
            if training:
                ds = ds.shuffle(buffer_size=min(2048, len(x)), seed=seed, reshuffle_each_iteration=True)
                ds = ds.map(lambda img, lab: (augment(img, training=True), lab),
                            num_parallel_calls=tf.data.AUTOTUNE)
            ds = ds.batch(batch, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
        return ds

# ---------- 3) باني النماذج للنسخ المختلفة ----------
def build_hyra_from_hp_variant(
    hp, 
    input_shape=(img_size, img_size, 3), 
    num_classes=len(categories),
    use_residual=True, 
    use_attention=True
):
    l2w          = float(hp.get("l2_weight"))
    drop         = float(hp.get("dropout"))
    f1           = int(hp.get("filters_s1"))
    f2           = int(hp.get("filters_s2"))
    f3           = int(hp.get("filters_s3"))
    head_filters = int(hp.get("head_filters"))
    opt_name     = str(hp.get("optimizer"))
    lr_val       = float(hp.get("lr"))

    reg = regularizers.l2(l2w)
    inputs = layers.Input(shape=input_shape)

    def conv_block(x, filters, k=3, s=1):
        x = layers.Conv2D(filters, k, strides=s, padding='same', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        return layers.ReLU()(x)

    def residual_block(x, filters):
        shortcut = x
        x = conv_block(x, filters)
        x = layers.Conv2D(filters, 3, padding='same', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same', kernel_regularizer=reg)(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        x = layers.Add()([shortcut, x])
        return layers.ReLU()(x)

    def attention_block(x, filters):
        # Channel attention
        w = layers.GlobalAveragePooling2D()(x)
        w = layers.Dense(max(filters // 8, 4), activation='relu')(w)
        w = layers.Dense(filters, activation='sigmoid')(w)
        x = layers.Multiply()([x, layers.Reshape((1,1,filters))(w)])
        # Spatial attention
        s = tf.reduce_mean(x, axis=-1, keepdims=True)
        s = layers.Conv2D(1, 7, padding='same', activation='sigmoid')(s)
        return layers.Multiply()([x, s])

    # ---- stem ----
    x = layers.Conv2D(f1, 3, strides=2, padding='same', kernel_regularizer=reg)(inputs)  # 112x112
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # RB1 (Residual أو بديله)
    if use_residual:
        x = residual_block(x, f1)
    else:
        x = conv_block(x, f1)
        x = layers.Conv2D(f1, 3, padding='same', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    x = layers.MaxPooling2D(2)(x)   # 56x56

    # RB2 + Attention-1
    if use_residual:
        x = residual_block(x, f2)
    else:
        x = conv_block(x, f2)
        x = layers.Conv2D(f2, 3, padding='same', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    if use_attention:
        x = attention_block(x, f2)
    x = layers.MaxPooling2D(2)(x)   # 28x28

    # RB3 + Attention-2
    if use_residual:
        x = residual_block(x, f3)
    else:
        x = conv_block(x, f3)
        x = layers.Conv2D(f3, 3, padding='same', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    if use_attention:
        x = attention_block(x, f3)
    x = layers.MaxPooling2D(2)(x)   # 14x14

    # ---- Head ----
    x = layers.Conv2D(head_filters, 1, activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(drop)(x)
    x = layers.Dense(256, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = models.Model(inputs, outputs, name=f"HyRA_variant_res{int(use_residual)}_att{int(use_attention)}")

    # Optimizer (مطابق لإعداداتك)
    if opt_name == "adamw":
        try:
            optimizer = tf.keras.optimizers.AdamW(learning_rate=lr_val)
        except Exception:
            optimizer = tf.keras.optimizers.experimental.AdamW(learning_rate=lr_val)
    elif opt_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_val)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_val)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def fresh_variant_model(best_hp, variant="no_attention"):
    if variant == "no_attention":
        return build_hyra_from_hp_variant(best_hp, use_residual=True,  use_attention=False)
    elif variant == "no_residual":
        return build_hyra_from_hp_variant(best_hp, use_residual=False, use_attention=True)
    else:
        raise ValueError("variant must be 'no_attention' or 'no_residual'")

# ---------- 4) حلقة التقييم 5-Fold وتخزين كل شيء تحت outputs2 ----------
def run_ablation_5fold(variant_name, hp, save_root="outputs2"):
    out_dir = os.path.join(save_root, f"ablation_{variant_name}")
    os.makedirs(out_dir, exist_ok=True)

    # احفظ نسخة من HPs المستخدمة
    with open(os.path.join(out_dir, "best_hps_used.json"), "w") as f:
        json.dump(hp.values, f, indent=2)

    y_all_cat = to_categorical(labels, len(categories)).astype('float32')
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    fold_scores = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(data, labels), start=1):
        print(f"\n===== Ablation [{variant_name}] FOLD {fold}/{FOLDS} =====")
        X_tr, X_va = data[tr_idx], data[va_idx]
        y_tr, y_va = y_all_cat[tr_idx], y_all_cat[va_idx]

        ds_train = make_ds(X_tr, y_tr, BATCH_SIZE, training=True,  seed=SEED)
        ds_val   = make_ds(X_va, y_va, BATCH_SIZE, training=False, seed=SEED)

        tf.keras.backend.clear_session(); gc.collect()

        model = fresh_variant_model(hp, variant=variant_name)

        cb_es  = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=40, restore_best_weights=True)
        cb_rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

        history = model.fit(
            ds_train,
            epochs=globals().get("FT_EPOCHS", 100),
            validation_data=ds_val,
            verbose=1,
            callbacks=[cb_es, cb_rlr]
        )

        # تقييم
        val_loss, val_acc = model.evaluate(ds_val, verbose=0)
        print(f"[{variant_name}] Fold-{fold} Accuracy: {val_acc*100:.2f}%")
        fold_scores.append(val_acc * 100)

        # حفظ history
        pd.DataFrame(history.history).to_csv(
            os.path.join(out_dir, f"training_history_{variant_name}_fold{fold}.csv"), index=False
        )

        # تقارير التصنيف
        y_true, y_pred = [], []
        for xb, yb in ds_val:
            preds = model.predict(xb, verbose=0)
            y_pred.extend(np.argmax(preds, axis=1))
            y_true.extend(np.argmax(yb.numpy(), axis=1))

        rep_df = pd.DataFrame(classification_report(y_true, y_pred, target_names=categories, output_dict=True)).transpose()
        rep_df.to_csv(os.path.join(out_dir, f"classification_report_{variant_name}_fold{fold}.csv"))

        # (اختياري) حفظ مصفوفة الالتباس كـ CSV
        # cm = confusion_matrix(y_true, y_pred)
        # pd.DataFrame(cm, index=categories, columns=categories).to_csv(
        #     os.path.join(out_dir, f"confusion_matrix_{variant_name}_fold{fold}.csv")
        # )

        tf.keras.backend.clear_session(); gc.collect()

    print(f"\n==== ABLATION SUMMARY [{variant_name}] ====")
    print("Accuracies per fold (%):", [f"{s:.2f}" for s in fold_scores])
    print(f"Mean Accuracy: {np.mean(fold_scores):.2f}%   ±   Std: {np.std(fold_scores):.2f}%")

    # حفظ ملخص الدقة
    pd.DataFrame({"fold": list(range(1, FOLDS+1)), "accuracy_percent": fold_scores}).to_csv(
        os.path.join(out_dir, f"summary_{variant_name}.csv"), index=False
    )

# ---------- 5) تشغيل تجارب الأبليشن وحفظ النتائج في outputs2 ----------
run_ablation_5fold("no_attention", best_hp)  # نموذج بدون Attention
#run_ablation_5fold("no_residual",  best_hp)  # نموذج بدون Residual
print("\nAll ablation artifacts saved under ./outputs2/")


Loaded: (3475, 224, 224, 3), Labels: (3475,)
Loaded best HPs from CSV: {'l2_weight': 5e-05, 'dropout': 0.3, 'filters_s1': 48, 'filters_s2': 96, 'filters_s3': 128, 'head_filters': 160, 'optimizer': 'rmsprop', 'lr': 0.001}
Best holdout val_accuracy (from CSV): 0.9353

===== Ablation [no_attention] FOLD 1/5 =====
Epoch 1/100
174/174 [==============================] - 14s 71ms/step - loss: 0.8621 - accuracy: 0.6327 - val_loss: 3.0610 - val_accuracy: 0.3165 - lr: 0.0010
Epoch 2/100
174/174 [==============================] - 10s 55ms/step - loss: 0.6392 - accuracy: 0.7482 - val_loss: 1.2260 - val_accuracy: 0.4576 - lr: 0.0010
Epoch 3/100
174/174 [==============================] - 10s 57ms/step - loss: 0.5669 - accuracy: 0.7835 - val_loss: 0.7135 - val_accuracy: 0.7079 - lr: 0.0010
Epoch 4/100
174/174 [==============================] - 10s 58ms/step - loss: 0.5344 - accuracy: 0.7975 - val_loss: 0.5614 - val_accuracy: 0.7957 - lr: 0.0010
Epoch 5/100
174/174 [==============================] - 1

Epoch 40/100
174/174 [==============================] - 12s 66ms/step - loss: 0.2309 - accuracy: 0.9263 - val_loss: 0.3298 - val_accuracy: 0.8921 - lr: 1.0000e-06
Epoch 41/100
174/174 [==============================] - 11s 65ms/step - loss: 0.2364 - accuracy: 0.9194 - val_loss: 0.3350 - val_accuracy: 0.8906 - lr: 1.0000e-06
Epoch 42/100
174/174 [==============================] - 12s 66ms/step - loss: 0.2421 - accuracy: 0.9227 - val_loss: 0.3313 - val_accuracy: 0.8921 - lr: 1.0000e-06
Epoch 43/100
174/174 [==============================] - 11s 66ms/step - loss: 0.2389 - accuracy: 0.9216 - val_loss: 0.3314 - val_accuracy: 0.8921 - lr: 1.0000e-06
Epoch 44/100
174/174 [==============================] - 12s 66ms/step - loss: 0.2424 - accuracy: 0.9255 - val_loss: 0.3304 - val_accuracy: 0.8935 - lr: 1.0000e-06
Epoch 45/100
174/174 [==============================] - 12s 66ms/step - loss: 0.2473 - accuracy: 0.9198 - val_loss: 0.3306 - val_accuracy: 0.8921 - lr: 1.0000e-06
Epoch 46/100
174/174 [

Epoch 24/100
174/174 [==============================] - 13s 73ms/step - loss: 0.2499 - accuracy: 0.9216 - val_loss: 0.2769 - val_accuracy: 0.9108 - lr: 3.1250e-05
Epoch 25/100
174/174 [==============================] - 12s 70ms/step - loss: 0.2413 - accuracy: 0.9158 - val_loss: 0.2733 - val_accuracy: 0.9108 - lr: 3.1250e-05
Epoch 26/100
174/174 [==============================] - 13s 72ms/step - loss: 0.2405 - accuracy: 0.9140 - val_loss: 0.2928 - val_accuracy: 0.9007 - lr: 3.1250e-05
Epoch 27/100
173/174 [============================>.] - ETA: 0s - loss: 0.2496 - accuracy: 0.9176
Epoch 27: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
174/174 [==============================] - 12s 68ms/step - loss: 0.2492 - accuracy: 0.9180 - val_loss: 0.2767 - val_accuracy: 0.9079 - lr: 3.1250e-05
Epoch 28/100
174/174 [==============================] - 12s 68ms/step - loss: 0.2392 - accuracy: 0.9187 - val_loss: 0.2712 - val_accuracy: 0.9065 - lr: 1.5625e-05
Epoch 29/100
174/174 [

174/174 [==============================] - 12s 70ms/step - loss: 0.2307 - accuracy: 0.9223 - val_loss: 0.2778 - val_accuracy: 0.9094 - lr: 1.0000e-06
Epoch 70/100
174/174 [==============================] - 12s 70ms/step - loss: 0.2289 - accuracy: 0.9187 - val_loss: 0.2772 - val_accuracy: 0.9079 - lr: 1.0000e-06
Epoch 71/100
174/174 [==============================] - 12s 71ms/step - loss: 0.2254 - accuracy: 0.9237 - val_loss: 0.2765 - val_accuracy: 0.9108 - lr: 1.0000e-06
Epoch 72/100
174/174 [==============================] - 12s 71ms/step - loss: 0.2332 - accuracy: 0.9241 - val_loss: 0.2774 - val_accuracy: 0.9108 - lr: 1.0000e-06
[no_attention] Fold-2 Accuracy: 91.37%

===== Ablation [no_attention] FOLD 3/5 =====
Epoch 1/100
174/174 [==============================] - 15s 70ms/step - loss: 0.8277 - accuracy: 0.6572 - val_loss: 5.2064 - val_accuracy: 0.3165 - lr: 0.0010
Epoch 2/100
174/174 [==============================] - 12s 68ms/step - loss: 0.6486 - accuracy: 0.7446 - val_loss: 1.8

174/174 [==============================] - 12s 68ms/step - loss: 0.2342 - accuracy: 0.9205 - val_loss: 0.3779 - val_accuracy: 0.8662 - lr: 1.0000e-06
Epoch 38/100
174/174 [==============================] - 12s 68ms/step - loss: 0.2331 - accuracy: 0.9237 - val_loss: 0.3757 - val_accuracy: 0.8691 - lr: 1.0000e-06
Epoch 39/100
174/174 [==============================] - 12s 68ms/step - loss: 0.2332 - accuracy: 0.9230 - val_loss: 0.3758 - val_accuracy: 0.8691 - lr: 1.0000e-06
Epoch 40/100
174/174 [==============================] - 12s 69ms/step - loss: 0.2322 - accuracy: 0.9273 - val_loss: 0.3758 - val_accuracy: 0.8691 - lr: 1.0000e-06
Epoch 41/100
174/174 [==============================] - 12s 69ms/step - loss: 0.2363 - accuracy: 0.9252 - val_loss: 0.3766 - val_accuracy: 0.8691 - lr: 1.0000e-06
Epoch 42/100
174/174 [==============================] - 12s 69ms/step - loss: 0.2378 - accuracy: 0.9155 - val_loss: 0.3776 - val_accuracy: 0.8691 - lr: 1.0000e-06
Epoch 43/100
174/174 [=============

173/174 [============================>.] - ETA: 0s - loss: 0.2810 - accuracy: 0.8970
Epoch 21: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
174/174 [==============================] - 12s 67ms/step - loss: 0.2805 - accuracy: 0.8975 - val_loss: 0.4184 - val_accuracy: 0.8417 - lr: 1.5625e-05
Epoch 22/100
174/174 [==============================] - 12s 67ms/step - loss: 0.2776 - accuracy: 0.9040 - val_loss: 0.3697 - val_accuracy: 0.8763 - lr: 7.8125e-06
Epoch 23/100
173/174 [============================>.] - ETA: 0s - loss: 0.2764 - accuracy: 0.9097
Epoch 23: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
174/174 [==============================] - 12s 67ms/step - loss: 0.2757 - accuracy: 0.9101 - val_loss: 0.3670 - val_accuracy: 0.8777 - lr: 7.8125e-06
Epoch 24/100
174/174 [==============================] - 12s 68ms/step - loss: 0.2774 - accuracy: 0.9032 - val_loss: 0.3791 - val_accuracy: 0.8647 - lr: 3.9063e-06
Epoch 25/100
173/174 [===============

174/174 [==============================] - 12s 66ms/step - loss: 0.3859 - accuracy: 0.8640 - val_loss: 0.4269 - val_accuracy: 0.8331 - lr: 5.0000e-04
Epoch 12/100
174/174 [==============================] - 12s 67ms/step - loss: 0.3699 - accuracy: 0.8673 - val_loss: 0.4622 - val_accuracy: 0.8374 - lr: 5.0000e-04
Epoch 13/100
173/174 [============================>.] - ETA: 0s - loss: 0.3701 - accuracy: 0.8721
Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
174/174 [==============================] - 12s 68ms/step - loss: 0.3688 - accuracy: 0.8727 - val_loss: 0.4448 - val_accuracy: 0.8245 - lr: 5.0000e-04
Epoch 14/100
174/174 [==============================] - 12s 66ms/step - loss: 0.3236 - accuracy: 0.8892 - val_loss: 0.4055 - val_accuracy: 0.8489 - lr: 2.5000e-04
Epoch 15/100
174/174 [==============================] - 12s 66ms/step - loss: 0.3189 - accuracy: 0.8878 - val_loss: 0.3340 - val_accuracy: 0.8633 - lr: 2.5000e-04
Epoch 16/100
174/174 [==============

174/174 [==============================] - 12s 68ms/step - loss: 0.2358 - accuracy: 0.9237 - val_loss: 0.3167 - val_accuracy: 0.8849 - lr: 1.0000e-06
Epoch 53/100
174/174 [==============================] - 12s 67ms/step - loss: 0.2427 - accuracy: 0.9216 - val_loss: 0.3198 - val_accuracy: 0.8835 - lr: 1.0000e-06
Epoch 54/100
174/174 [==============================] - 12s 67ms/step - loss: 0.2452 - accuracy: 0.9158 - val_loss: 0.3156 - val_accuracy: 0.8849 - lr: 1.0000e-06
Epoch 55/100
174/174 [==============================] - 12s 67ms/step - loss: 0.2331 - accuracy: 0.9234 - val_loss: 0.3139 - val_accuracy: 0.8878 - lr: 1.0000e-06
Epoch 56/100
174/174 [==============================] - 12s 68ms/step - loss: 0.2416 - accuracy: 0.9277 - val_loss: 0.3136 - val_accuracy: 0.8878 - lr: 1.0000e-06
Epoch 57/100
174/174 [==============================] - 12s 69ms/step - loss: 0.2438 - accuracy: 0.9216 - val_loss: 0.3167 - val_accuracy: 0.8849 - lr: 1.0000e-06
Epoch 58/100
174/174 [=============